In [3]:
from Bio import SearchIO
from Bio.SeqIO import *

from tst.domutil.util import *
from tst.utils_db import *


os.environ["SEQlib"] = full("$DATA/sequence-data")

In [4]:
# reload(tst.domutil.util)
%load_ext autoreload
%autoreload 0

In [2]:
### Manually generated tmp.hmmpf and tmp.domtbl

### search_
wkdir = "/data/sequence-data/"
hmm_profile = "%s/tmp.hmmpf" % wkdir
hmm_domtbl = "%s/tmp.domtbl" % wkdir
# CATH_S35

# kwargs = {"format":"hmmer3-domtab"}
tbl_format = "hmmsearch3-domtab"

In [3]:
import os
os.environ["SEQlib"]

'/data/sequence-data'

## import sequence database

In [82]:
debug = 1


behave = "import sequence database"
seqdb_file = full("$SEQlib/cath-domain-seqs-S100.fa")

from time import time
t0 = time()
useFisrt = 1
failcount = 0

with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)
    c = counter(iterator, per=1000)
with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)

    with transaction.atomic():
        for values in iterator:
            try:
#             if 1:
                header,seq = values
                if c.i == 0:
                    (h_parser, seqDB_curr) = seqheader_guess_parser(header)
                    print "first entry pointing to %s" % seqDB_curr
                    print "h_parser is",h_parser
                    
                jdict = h_parser( header )
                jdict["length"] = len(seq)
                if useFisrt:
                    jdict["seqDB"] = seqDB_curr
                else:
                    jdict["seqDB"] = seqDB.objects.get(**jdict["seqDB"])
                seqobj = sequence(**jdict)
                seqobj.save()
            except Exception as e:
                if debug: print >>sys.__stdout__,e,header
                failcount += 1
            c.count()
        
#         print(values)
print "finshed %s from %s" % (behave, seqdb_file)

print '%d instances of %d failed' % (failcount,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = failcount/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate

first entry pointing to CATH_Ver:v4_1_0
h_parser is <function seqheader_parse_cath at 0x7fcdc40de398>
finshed import sequence database from /data/sequence-data/cath-domain-seqs-S100.fa
0 instances of 80422 failed
Ended after 56.2935 sec


## Importing HMM profiles from .lib file

In [51]:
%autoreload 0


In [49]:
seqheader_parse_cath(p_header.findall(hmm_text)[0]).get("acc")

'16pkA01'

In [50]:
from time import time
from tst.domutil.util import *
import re
p_header = re.compile("NAME.*?\n")
# from copy import copy
hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
behave = "importing HMM profiles"


t0 = time()
lst = parse_hmmlib( hmmlib_file )
c = counter( lst)
lst = parse_hmmlib( hmmlib_file )
failcount = 0

try:
    with transaction.atomic():
        for hmm_text in lst:
            c.count()
            acc = seqheader_parse_cath(
                p_header.findall(hmm_text)[0]
            ).get("acc")
#             acc = next(header_gen)
            try:
                cnode = domain.objects.get(domain_id = acc).classification
                if cnode.hmmprofile_set.exists():
                    pass
                else:
                    hmm = HMMprofile(
                                cath_node_id = cnode.id,
                                text = hmm_text,               
                                )
                    hmm.save()
            except Exception as e:
                print "failed for %s for Exception:%s" %( acc, e)
                failcount += 1
except Exception as e:
    print "early stop due to %s" % e

print "finshed %s from %s" % (behave, hmmlib_file)

print '%d instances of %d failed' % (failcount,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = failcount/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate

get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one domain -- it returned 2!
get() returned more than one

## HMMsearch all the loaded HMM profiles (against CATHS100)

In [4]:
%load_ext autoreload
%autoreload 0

In [16]:
# import psutil
current_process # current_process = psutil.Process()
# children = current_process.children(recursive=True)
# for child in children:
#     print('Child pid is {}'.format(child.pid))

<psutil.Process(pid=25578, name='python') at 140139357836176>

In [6]:
from time import time
from tst.domutil.util import *
import re
import gc
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
behave = "HMMsearching profiles"

HMMprofile.objects.all()

t0 = time()

qset = HMMprofile.objects.order_by("id")
c = counter(range(qset.count()), per = 100)
batches = batch_qs(qset, batch_size =500)
# it = lst.iterator()
failcount = 0

# print "hi"
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/tmp2"                
    )

    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp, query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id

        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()

transaction.set_autocommit( False)
if 1:
# with transaction.atomic():
    for batch in batches:
        for hmm in batch:
            c.count()
            if c.i < 500:
                continue
            try:
                worker()
#                 hits = hmmsearch(hmm,
#                 seqDB_curr = seqDB_curr,
#                 seqDB_file = seqDB_file,
#                 )
            except Exception as e:
                c.fail("failed for %s for %s" % (hmm,e))
#                 failcount += 1
            if not c.i % c.per:
                transaction.commit()



print "finshed %s from %s" % (behave, hmmlib_file)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate
transaction.set_autocommit(True)


0 of 21078
100 of 21078
200 of 21078
300 of 21078
400 of 21078
500 of 21078
600 of 21078
failed for HMM for 3.90.226.10.3.0  for get() returned more than one sequence -- it returned 2!
700 of 21078
800 of 21078


tst/domutil/util.py:319: RuntimeWarning: divide by zero encountered in log10
  jdict["logCevalue"] = np.log10(jdict.pop("evalue_cond"))
tst/domutil/util.py:320: RuntimeWarning: divide by zero encountered in log10
  jdict["logIevalue"] = np.log10(jdict.pop("evalue"))


failed for HMM for 3.20.20.350.1.0  for (1054, "Unknown column 'inf' in 'where clause'")
failed for HMM for 3.90.226.10.46.0  for get() returned more than one sequence -- it returned 2!
900 of 21078
1000 of 21078
1100 of 21078
failed for HMM for 3.60.20.10.29.0  for get() returned more than one sequence -- it returned 2!
1200 of 21078
1300 of 21078
1400 of 21078
failed for HMM for 3.20.70.20.1.0  for (1054, "Unknown column 'inf' in 'where clause'")
1500 of 21078
failed for HMM for 3.90.226.10.62.0  for get() returned more than one sequence -- it returned 2!
1600 of 21078
1700 of 21078
1800 of 21078
1900 of 21078
failed for HMM for 1.20.1130.10.1.0  for (1054, "Unknown column 'inf' in 'where clause'")
2000 of 21078
2100 of 21078
2200 of 21078
2300 of 21078
2400 of 21078
2500 of 21078
2600 of 21078
failed for HMM for 3.90.1290.10.2.0  for get() returned more than one sequence -- it returned 2!
2700 of 21078
2800 of 21078
2900 of 21078
3000 of 21078
3100 of 21078
3200 of 21078
failed for 

KeyboardInterrupt: 

In [10]:
# # import re
# # p_header = re.compile("NAME.*?\n")
# # p_header.findall(lst[2].text)
#             hits = hmmsearch(hmm.text,
#             seqDB_curr = seqDB_curr,
#             seqDB_file = seqDB_file,
#             )
# hmm = lst[0]
# len(lst)
lst.count()

21078

In [4]:
# batch_qs
# lst.order_by("id")

<QuerySet [<HMMprofile: HMM for 2.30.40.10.24.0 >, <HMMprofile: HMM for 3.40.50.300.361.0 >, <HMMprofile: HMM for 2.40.10.10.8.0 >, <HMMprofile: HMM for 2.40.10.10.16.0 >, <HMMprofile: HMM for 3.30.1120.60.1.0 >, <HMMprofile: HMM for 2.60.120.200.50.0 >, <HMMprofile: HMM for 2.80.10.50.72.0 >, <HMMprofile: HMM for 3.40.30.10.74.0 >, <HMMprofile: HMM for 3.40.30.10.125.0 >, <HMMprofile: HMM for 1.10.1200.30.2.0 >, <HMMprofile: HMM for 1.20.20.10.4.0 >, <HMMprofile: HMM for 4.10.220.40.1.0 >, <HMMprofile: HMM for 3.80.10.10.34.0 >, <HMMprofile: HMM for 3.30.1490.20.5.0 >, <HMMprofile: HMM for 1.10.1030.10.1.0 >, <HMMprofile: HMM for 3.40.50.20.8.0 >, <HMMprofile: HMM for 3.30.470.20.7.0 >, <HMMprofile: HMM for 3.40.50.1380.8.0 >, <HMMprofile: HMM for 3.50.30.20.1.0 >, <HMMprofile: HMM for 3.40.50.880.31.0 >, '...(remaining elements truncated)...']>

In [6]:
# del hmmsearch.__module__
del hmmsearch


In [4]:
import os
os.environ["SEQlib"]

KeyError: 'SEQlib'

In [18]:
# %load_ext reload
# del tst.domutil.util
%autoreload 2


In [ ]:
parser = SearchIO.parse( INPUT, tbl_format )
c = counter(parser)


In [ ]:
# parser = SearchIO.parse( INPUT, tbl_format )


In [5]:
## Load HMMsearch results from an external file

from time import time
from tst.domutil.util import *
from tst.utils_db import *
from Bio import SearchIO
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))


seqDB_curr = seqDB.objects.get(name = 'CATH')
# seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
INPUT = "/media/shouldsee/DATA_FENG/data/archive/CATHS100_v410.domtbl"
tbl_format = 'hmmsearch3-domtab'
hmm_resultfile = INPUT
behave = "Loading HMMsearch results"
t0 = time()

parser = SearchIO.parse( INPUT, tbl_format )
c = counter(parser, INF = True)
parser = SearchIO.parse( INPUT, tbl_format )

# if 1:
with transaction.atomic():
    for q_hits in parser:
#         q_hits = next(parser)
    #     q_hits = next(parser)    
        try:
#         if 1:
            q_hits.id = seqheader_parse_cath(q_hits.id)["acc"]
            hmm = domain.objects.get(domain_id = q_hits.id).classification.hmmprofile_set.first()
            q_hits = q_hits.hit_map(acc_mapper_cath)

            oldhits = hmm.hit4hmm2hsp_set.all()
            for hit in q_hits:
                hsp = hit[0] ### Assume only one dom per hit
                jdict = hsp2jdict(hsp, query = hmm )
                jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id
                
                if not oldhits.filter(**jdict).exists():
                    hit_db = hit4hmm2hsp(**jdict)
                    hit_db.save()
#         try:
#             pass
        except Exception as e:
            c.fail("Failed at %s for Exception:%s" % (hmm, e))
        finally:
            c.count()
    
            
c.finish()

print "finshed %s from %s" % (behave, INPUT)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate


0 of -1


KeyboardInterrupt: 

In [45]:
print "%d" % None

TypeError: %d format: a number is required, not NoneType

In [38]:
# domain.objects.get(domain_id = q_hits.id)
# q_hits
hmm
# domain.objects.count()

<HMMprofile: HMM for 3.40.50.1260.1.0 >

In [12]:
hmmpf_file = full("$SEQlib/cath-S35-hmm3.lib")
# hmmpf_file = full("$SEQlib/tmp.hmmpf")

def search_idx(q_acc, fname, acc_func):
    with open(fname, 'r') as f:
        line = f.readline()
        idx = 0
        while line:
#             header = line.rstrip("\n")[6:]
#             acc = seqheader_parse_cath(header)["acc"]
            ac = acc_func(line)
            if acc == q_acc:
                break
            
            line = f.readline()
            idx += 1
        if not line:
            return None
        else:
            return idx
        
        

        
import subprocess
def search_lib( q_acc, fname = hmmpf_file, acc_func = None):
    
    ##### check whether an index file exists *.idx
    idxfile = fname + '.idx'
    if not os.path.isfile(idxfile):
        subprocess.call(['cat',fname,'|','grep','NAME','>',idxfile])
    if not acc_func:
        acc_func = lambda line: seqheader_parse_cath(line.rstrip("\n")[6:])["acc"]
    
    idx = search_idx(q_acc ,idxfile, acc_func)
    print idx

    if not idx:
        return None
    else:
        with open(fname,'r') as f:
            line = 1
            cidx = 0
#             ln = 0
            buf = ""
            while line:
                line = f.readline()                
                if cidx == idx:
                    buf += line
                if line.rstrip('\n')=='//':
                    cidx += 1
                    if buf:
                        return buf + line


# p_acc = "2y8gB00"
# dom = domain.objects.get(domain_id = p_acc)
# print search_lib( p_acc )        
# subprocess.call(['hmmscan', '--domtblout', 'output.txt', '/location/to/profile.hmm', 'input.proteins.fasta']

### import scanning resultv
                

In [5]:
class s35():
    def __init__(self, domain_id, hmmlib_file = None):
        try:
            self.domain = domain.objects.get( domain_id = domain_id )
            self.node = self.domain.classification
        except Exception as e: 
            self.domain = None
            self.node = None
            print "failed to init s35 for exception: %s" % e
        if self.node:
            if hasattr(self.node,'HMMprofile'):
                self.hmm = self.node.HMMprofile
            else:
                hmm_text = search_lib(domain_id, hmmlib_file)
                hmm = HMMprofile(
                    cath_node_id = self.node.id,
                    text = hmm_text,               
                    )
                hmm.save()
                self.hmm = hmm 
        else:
            print "not creating hmm profile in lack of hierarchy"
    def hmmsearch(self,**kwargs):
        self.hitset =  hmmsearch(self.hmm ,**kwargs)
        
#         return

if __name__=="__main__":
    s35A = s35( "2q09A01", hmmlib_file = hmmpf_file)
def hsp2jdict(hsp,query = None):
    jdict = hsp.__dict__
    jdict["query_id"] = query.id
#     print hsp.hit_id
    jdict["target_id"] = sequence.objects.get( acc = hsp.hit_id).id
    it = jdict.pop('_items')
    jdict.pop('domain_index')
    jdict["start"]=jdict.pop("env_start")
    jdict["end"]=jdict.pop("env_end")
    jdict["logCevalue"] = np.log10(jdict.pop("evalue_cond"))
    jdict["logIevalue"] = np.log10(jdict.pop("evalue"))
    
    return jdict

# import gc
# def hmmsearch(hmm,seqDB_curr = None, seqDB_file = None, tmpdir = "/tmp"):
# if 1:
#     tmpdir = "/tmp"
#     hmm = s35A.hmm
#     seqDB_file = full("$SEQlib/cath-domain-seqs-S100.fa")
def hmmsearch(hmm,seqDB_curr = None, seqDB_file = None, tmpdir = "/tmp"):
#     if ["seqDB_curr"] in locals().keys():
    if not seqDB_curr:
        seqDB_curr = seqDB.objects.get(name = 'CATH')
#         seqDB_file = seqDB_curr.filepath
    if seqDB_file:
        seqDB_file = full(seqDB_file)
    def acc_mapper(hit):
#         hit.query
        hit.id = seqheader_parse_cath(hit.id)["acc"]
        return hit    

    tmphmm = "%s/hmm" % tmpdir
    with open(tmphmm,'w') as f:
        f.write(hmm.text)
    cmd = ["hmmsearch","--noali","-o",tmpdir+"/log","--domtblout",tmpdir + "/domtbl", 
                                     tmphmm, seqDB_file,]
#     print cmd
    qtext = subprocess.check_output( cmd )
    gc.collect()

    parser = SearchIO.parse( tmpdir + "/domtbl", tbl_format )
    q_hits = next(parser)
    q_hits.id = seqheader_parse_cath(q_hits.id)["acc"]

    q_hits = q_hits.hit_map(acc_mapper)

    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp, query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id
        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()
#     hmm.hits.filter()
#     dis_q = hmm.hit4hmm2hsp_set.distinct()

#     dis_q = hmm.hit4hmm2hsp_set.distinct("target")
    dis_q = hmm.hit4hmm2hsp_set.values_list("target",flat = True).distinct()
    
    q = hmm.hit4hmm2hsp_set.all()

    if dis_q.count() < q.count():
        q = q.exclude(id__in=list(dis_q.values_list("id",flat = True)) )
#         q.delete()
#         dis_q.update()
        
    return hmm.hit4hmm2hsp_set

10234


In [158]:
s35A = s35( "2q09A01", hmmlib_file = hmmpf_file)
s35B = s35( "8ohmA03", hmmlib_file = hmmpf_file)
print "finished"

with transaction.atomic():
    s35A.hmmsearch(
        seqDB_curr = seqDB.objects.get(name = 'CATH'),
        seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa",
    )
print s35A.hitset.count()

with transaction.atomic():
    s35B.hmmsearch(
        seqDB_curr = seqDB.objects.get(name = 'CATH'),
        seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa",
    )
print s35B.hitset.count()

print hit4hmm2hsp.objects.count()

10234
21154
finished
